## Imports

In [31]:
from datetime import date
import os.path

## Global Variables

In [32]:
patients_ids_list = []
patients_list = []
gurneys = []
number_of_occupied_gurneys = 0

## Classes

### Gurney Class

In [33]:
class Gurney:
    def __init__(self, id_gurney, occupied, id_patient):
        self.id_gurney = id_gurney
        self.occupied = occupied
        self.id_patient = id_patient
    
    def __str__(self):
        return "id_gurney: {}, occupied: {}, id_patient: {}".format(self.id_gurney, self.occupied, self.id_patient)
    
    def register_patient_on_gurney(self, gurneys, new_patient):
        for gurney in gurneys:
            if(gurney.occupied):
                number_of_occupied_gurneys += 1
        
        number_of_available_gurneys = (len(gurneys) - number_of_occupied_gurneys)
        
        if(number_of_available_gurneys > 0):
            for gurney in gurneys:
                if(gurney.occupied == False):
                    gurney.occupied = True
                    number_of_occupied_gurneys =+ 1
                    patients_list.append(new_patient)
                    
                    # update patient file
                    register_patients_on_files(new_patient)
                    
                    # update gurney file
                    register_gurneys_on_files(gurney)
                    
        else:
            print("There's no availiable gurneys...")

### Medical History Class

In [34]:
class MedicalHistory:
  vital_signs = {"blood pressure", "temperature", "o2 saturation", "respiratory rate"}
  evolution_note = ""
  laboratory_test_results = ""
  formulated_drugs = ""
  id_patient = ""
  name_patient = ""

  def __init__(self, patient_id=None, blood_pressure=None, temperature=None, o2_saturation=None, respiratory_rate=None, evolution_note=None, diagnosis=None, medicines_formuled=None):
    self.patient_id = patient_id
    self.blood_pressure = blood_pressure
    self.temperature = temperature
    self.o2_saturation = o2_saturation
    self.respiratory_rate = respiratory_rate
    self.evolution_note = evolution_note
    self.diagnosis = diagnosis
    self.medicines_formuled = medicines_formuled
    
  def __str__(self):
    return "patient_id: {}, blood_pressure: {}, temperature: {}, o2_saturation: {}, respiratory_rate: {}, evolution_note: {}, diagnosis: {}, medicines_formuled: {}".format(self.patient_id, self.blood_pressure, self.temperature, self.o2_saturation, self.respiratory_rate, self.evolution_note, self.diagnosis, self.medicines_formuled)

  def get_clinical_history(self):
    self.vital_signs = {}
    self.evolution_note = str(input("Enter the results of laboratory tests: "))
    self.blood_pressure = float(input("Enter your blood pressure: "))
    self.temperature = float(input("enter your temperature: "))
    self.o2_saturation = float(input("Enter your oxygen saturation: "))
    self.respiratory_rate = float(input("Enter your respiratory rate: "))
    self.evolution_note = str(input("Enter your evolution note: "))
    self.medicines_formuled = str(input("Enter your formulated medicines (separated by commas): "))

  def diagnosis_method(self):
    chronic_diseases = []
    if self.blood_pressure > 140/90:
      chronic_diseases.append("Arterial hypertension")
    if self.temperature > 37.5:
      chronic_diseases.append("Fever")
    if self.temperature < 35:
      chronic_diseases.append("Hypothermia")
    if self.o2_saturation < 95:
      chronic_diseases.append("Low oxygen saturation")
    if self.respiratory_rate < 12 or self.respiratory_rate > 20:
      chronic_diseases.append("Abnormal respiratory rate")
    self.diagnosis = chronic_diseases
    
    register_medical_history_on_files(self)
    print(chronic_diseases)

### Patient Class

In [35]:
class Patient:
  def __init__(self, id=None, name=None, sex=None, birth_day=None):
    self.id = id
    self.name = name
    self.sex = sex
    self.birth_day = birth_day
  
  def __str__(self):
    return "id: {}, name: {}, sex: {}, birth_day: {}".format(self.id, self.name, self.sex, self.birth_day)
  
  def get_patient_data(self, id="", name="", sex="", birth_day=""):
    
    self.id = str(input("Type the identification document of the patient: ")) #Valdiate if there's a patient with the entered id
    
    while(validate_patient_id(self.id) == "error"):
      print("There's already a patient registered with this identification...")
      self.id = str(input("Type the identification document of the patient: "))
    
    self.name = str(input("Type the patient's name: "))
    user_input_patient_sex = str(input("Type the patient's sex (M (male) | F (female)): "))
    
    # Validate that the sex is valid
    while (validate_patient_sex(user_input_patient_sex) == "error"):
      print("The unique valid options are M (male) or F (female)...")
      user_input_patient_sex = str(input("Type the patient's sex (male (M) | female (F)): "))
    self.sex = user_input_patient_sex
    
    # Get the patient's birthday
    self.birth_day = get_patient_birthday()
    
    # Update the patients file with the new patient
    register_patients_on_files(self)
    
    # Assign a gurney to the new patient
    assign_gurney_to_new_patient(self)

## Global functions

### Create a gurney-patient record on the gurneys file

In [36]:
def register_gurneys_on_files(new_gurney):  
  try:
    # Check if the file exists
    with open("./gurneys.txt", 'r') as file:
        existing_content = file.read()
      
    # If the file exists, append the line to the end
    with open("./gurneys.txt", 'a') as file:
      file.write("\n" + "id_gurney: {}, occupied: {}, id_patient: {}".format(new_gurney.id_gurney, new_gurney.occupied, new_gurney.id_patient))
    
    file.close()
  except FileNotFoundError:
    # If the file doesn't exist, create it and add the line at the beginning
    with open("./gurneys.txt", 'w') as file:
      file.write("id_gurney: {}, occupied: {}, id_patient: {}".format(new_gurney.id_gurney, new_gurney.occupied, new_gurney.id_patient))
    file.close()
  
    

### Create a medical history record on the medical histories file

In [37]:
def register_medical_history_on_files(new_medical_history):
  
  try:
    # Check if the file exists
    with open("./medical_histories.txt", 'r') as file:
        existing_content = file.read()
      
    # If the file exists, append the line to the end
    with open("./medical_histories.txt", 'a') as file:
      file.write("\n" + "patient_id: {}, blood_pressure: {}, temperature: {}, o2_saturation: {}, respiratory_rate: {}, evolution_note: {}, diagnosis: {}, medicines_formuled: {}".format(new_medical_history.patient_id, new_medical_history.blood_pressure, new_medical_history.temperature, new_medical_history.o2_saturation, new_medical_history.respiratory_rate, new_medical_history.evolution_note, new_medical_history.diagnosis, new_medical_history.medicines_formuled))
    file.close()
  except FileNotFoundError:
    # If the file doesn't exist, create it and add the line at the beginning
    with open("./medical_histories.txt", 'w') as file:
      file.write("patient_id: {}, blood_pressure: {}, temperature: {}, o2_saturation: {}, respiratory_rate: {}, evolution_note: {}, diagnosis: {}, medicines_formuled: {}".format(new_medical_history.patient_id, new_medical_history.blood_pressure, new_medical_history.temperature, new_medical_history.o2_saturation, new_medical_history.respiratory_rate, new_medical_history.evolution_note, new_medical_history.diagnosis, new_medical_history.medicines_formuled))
    file.close()

### Create a patient record on the patients file

In [38]:
def register_patients_on_files(new_patient):   
  try:
    # Check if the file exists
    with open("./patients.txt", 'r') as file:
        existing_content = file.read()
      
    # If the file exists, append the line to the end
    with open("./patients.txt", 'a') as file:
      file.write("\n" + "id: {}, name: {}, sex: {}, birth_day: {}".format(new_patient.id, new_patient.name, new_patient.sex, new_patient.birth_day))
    file.close()
  except FileNotFoundError:
    # If the file doesn't exist, create it and add the line at the beginning
    with open("./patients.txt", 'w') as file:
      file.write("id: {}, name: {}, sex: {}, birth_day: {}".format(new_patient.id, new_patient.name, new_patient.sex, new_patient.birth_day))
    file.close()

### Initialize Gurney Function

In [39]:

def initialize_gurney():
    for i in range(1, 301):
        id_gurney_temp = ""
        if i < 10:
            id_gurney_temp = ("gur00" + str(i))
        elif i >= 10 or i < 100:
            id_gurney_temp = ("gur0" + str(i))
        else:
            id_gurney_temp = ("gur" + str(i))
        occupied_temp = False
        id_patient_temp = "none"
        gurney = Gurney(id_gurney_temp, occupied_temp, id_patient_temp)
        register_gurneys_on_files(gurney)
        gurneys.append(gurney)

### Validate Patient's Id Function

In [40]:
def validate_patient_id(patient_id):
  
  try:
    # Check if the file exists
    with open("./patients.txt", 'r') as file:
        existing_content = file.read()
        if(existing_content.find(patient_id)):
          return "error"
  except FileNotFoundError:
    return "success"
  return "success"

### Validate Sex Function

In [41]:
def validate_patient_sex(sex):
  if(sex != "M" and sex != "m" and sex != "F" and sex != "f"):
    return "error"
  return "success"

### Get Patient's Birthday Function

In [42]:
def get_patient_birthday():
  print("Type the number of the patient birthday...")
  day = int(input("Day: "))
  month = int(input("Month: "))
  year = int(input("Year: "))
  return date(year,month,day)

### Assign a gurney to the new patient

In [43]:
def assign_gurney_to_new_patient(new_patient):
  try:
    updated_list = []
    # Read the original file and store its lines
    with open("./gurneys.txt", 'r') as file:
      lines = file.readlines()
    file.close()
    
    updated_list = lines
    # Update the available gurney
    for line in lines:
      if("occupied: False" in line):
        index = updated_list.index(line)
        line = line.replace("False", "True")
        line = line.replace("none", new_patient.id)
        updated_list[index] = line
        print(line)
        break
    print(lines[0])
    # Open the file for writing
    with open("./gurneys.txt", 'w') as file:
      for line in updated_list:
        file.write(line)
    file.close()
    
  except FileNotFoundError:
    print("File not found.")

### Assign a clinic history to the new patient

In [44]:
def assign_clinic_history_to_patient():
  try:
    # Check if the file exists
    with open("./patients.txt", 'r') as file:
        existing_content = file.read()
      
    # If the file exists, append the line to the end
    with open("./patients.txt", 'a') as file:
      file.write("\n" + "id: {}, name: {}, sex: {}, birth_day: {}".format(new_patient.id, new_patient.name, new_patient.sex, new_patient.birth_day))
    file.close()
  except FileNotFoundError:
    # If the file doesn't exist, create it and add the line at the beginning
    with open("./patients.txt", 'w') as file:
      file.write("id: {}, name: {}, sex: {}, birth_day: {}".format(new_patient.id, new_patient.name, new_patient.sex, new_patient.birth_day))
    file.close()

### Delete pacient from the guerney because it's recovered

In [ ]:
def delete_patient_from_gurney(patient_id):
  try:
    patient_found = False
    updated_list = []
    # Read the original file and store its lines
    with open("./gurneys.txt", 'r') as file:
      lines = file.readlines()
    file.close()
    updated_list = lines
    
    # Update the available gurney
    for line in lines:
      if(patient_id in line):
        index = updated_list.index(line)
        line = line.replace("True", "False")
        line = line.replace(patient_id, "none")
        updated_list[index] = line
        patient_found = True
    # Open the file for writing
    with open("./gurneys.txt", 'w') as file:
      for line in updated_list:
        file.write(line)
    file.close()
    
    if(patient_found == False):
      return "not found"
    else:
      return "found"
    
  except FileNotFoundError:
    print("File not found.")

### Get Patient Reports

In [ ]:
def get_patient_reports():
  with open("./gurneys.txt", 'r') as file:
    gurneys_content = file.read()
  file.close()
  
  with open("./patients.txt", 'r') as file:
    patients_content = file.read()
  file.close()
  
  with open("./medical_histories.txt", 'r') as file:
    medical_histories_content = file.read()
  file.close()
  
  amount_of_occupied_guernys = 0
  
  for line in gurneys_content:
    if("True" in line):
      amount_of_occupied_guernys += 1
  
  print("************************** Percentage of bed occupancy **************************")
  print((amount_of_occupied_guernys * 100)/300, "%")
  
  print("************** Number of admissions and discharges by service *******************")
  print("Admissions: " + amount_of_occupied_guernys)
  print("Hospital discharges")
  
  print("*********************** Patients with chronic diseases **************************")
  for line in medical_histories_content:
    if("Arterial hypertension" in line or "Fever" in line or "Hypothermia" in line or "Low oxygen saturation" in line or "Abnormal respiratory rate" in line):
      print(line)
      
  print("*********************** Medicines per service**************************")
  for line in medical_histories_content:
    if("Arterial hypertension" in line or "Fever" in line or "Hypothermia" in line or "Low oxygen saturation" in line or "Abnormal respiratory rate" in line):
      print(line[0:line.find(",")] + " Chronic Diseases"  + line[line.find("["):s.find("]")])

### Main Function

In [45]:
if __name__ == "__main__":
  initialize_gurney()
  keep_option = "Y"
  while(option == "Y"):
    print("************************** WELCOME **************************")
    print("1. Add a new patient")
    print("2. Discharged a patient")
    print("3. Watch patients reports")
    print("4. Exit")
    print("*************************************************************")
    option = int(input("What option do you want?: "))
    
    while(option < 1 or option > 4):
      print("INVALID OPTIOM...")
      option = int(input("What option do you want?: "))
    
    if(option == "1"):
      new_patient = Patient()
      new_patient.get_patient_data()
      new_patient_clinic_history = MedicalHistory(new_patient.id)
      new_patient_clinic_history.get_clinical_history()
      new_patient_clinic_history.diagnosis_method()
      print("***************** Patient Data ******************************")
      print(new_patient)
      print(new_patient_clinic_history)
      
    elif(option == "2"):
      patient_id_to_release = str(input("Type the document of the patient the patient to be discharged: "))
      response = delete_patient_from_gurney(patient_id_to_release)
      if(response == "not found"):
        print("There's no patient with that document")
      else:
        print("Patient discharged")
    elif(option == "3"):
      get_patient_reports
    else:
      keep_option = "N"
    
    
    
  
    

Type the number of the patient birthday...
id_gurney: gur001, occupied: True, id_patient: 123

id_gurney: gur001, occupied: True, id_patient: 123

['Arterial hypertension', 'Fever', 'Low oxygen saturation', 'Abnormal respiratory rate']
id: 123, name: dan, sex: m, birth_day: 1999-01-01
